In [16]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import imgaug

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

### Configuración

In [2]:
DEVICE = "/gpu:0"

In [3]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [19]:
import coco
config = coco.CocoConfig()
config.IMAGE_META_SIZE = 73
COCO_DIR = "dataset"
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                73
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

### Carga de datos

In [8]:
#Training dataset
dataset_train = coco.CocoDataset()
dataset_train.load_coco(COCO_DIR, "train")
dataset_train.prepare()

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!


In [9]:
print("Images: {}\nClasses: {}".format(len(dataset_train.image_ids), dataset_train.class_names))

Images: 4380
Classes: ['BG', 'adelholzener_alpenquelle_classic_075', 'adelholzener_alpenquelle_naturell_075', 'adelholzener_classic_bio_apfelschorle_02', 'adelholzener_classic_naturell_02', 'adelholzener_gourmet_mineralwasser_02', 'augustiner_lagerbraeu_hell_05', 'augustiner_weissbier_05', 'coca_cola_05', 'coca_cola_light_05', 'suntory_gokuri_lemonade', 'tegernseer_hell_03', 'corny_nussvoll', 'corny_nussvoll_single', 'corny_schoko_banane', 'corny_schoko_banane_single', 'dr_oetker_vitalis_knuspermuesli_klassisch', 'koelln_muesli_fruechte', 'koelln_muesli_schoko', 'caona_cocoa', 'cocoba_cocoa', 'cafe_wunderbar_espresso', 'douwe_egberts_professional_ground_coffee', 'gepa_bio_caffe_crema', 'gepa_italienischer_bio_espresso', 'apple_braeburn_bundle', 'apple_golden_delicious', 'apple_granny_smith', 'apple_red_boskoop', 'avocado', 'banana_bundle', 'banana_single', 'clementine', 'clementine_single', 'grapes_green_sugraone_seedless', 'grapes_sweet_celebration_seedless', 'kiwi', 'orange_single', 

In [10]:
#Validation dataset
dataset_val = coco.CocoDataset()
dataset_val.load_coco(COCO_DIR, "val")
dataset_val.prepare()

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!


In [11]:
print("Images: {}\nClasses: {}".format(len(dataset_val.image_ids), dataset_val.class_names))

Images: 3600
Classes: ['BG', 'adelholzener_alpenquelle_classic_075', 'adelholzener_alpenquelle_naturell_075', 'adelholzener_classic_bio_apfelschorle_02', 'adelholzener_classic_naturell_02', 'adelholzener_gourmet_mineralwasser_02', 'augustiner_lagerbraeu_hell_05', 'augustiner_weissbier_05', 'coca_cola_05', 'coca_cola_light_05', 'suntory_gokuri_lemonade', 'tegernseer_hell_03', 'corny_nussvoll', 'corny_nussvoll_single', 'corny_schoko_banane', 'corny_schoko_banane_single', 'dr_oetker_vitalis_knuspermuesli_klassisch', 'koelln_muesli_fruechte', 'koelln_muesli_schoko', 'caona_cocoa', 'cocoba_cocoa', 'cafe_wunderbar_espresso', 'douwe_egberts_professional_ground_coffee', 'gepa_bio_caffe_crema', 'gepa_italienischer_bio_espresso', 'apple_braeburn_bundle', 'apple_golden_delicious', 'apple_granny_smith', 'apple_red_boskoop', 'avocado', 'banana_bundle', 'banana_single', 'clementine', 'clementine_single', 'grapes_green_sugraone_seedless', 'grapes_sweet_celebration_seedless', 'kiwi', 'orange_single', 

### Entrenamiento

In [20]:
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="training", model_dir=MODEL_DIR,
                              config=config)

In [21]:
# Load weights
print("Loading weights ", COCO_MODEL_PATH)
model.load_weights(COCO_MODEL_PATH, by_name=True)

Loading weights  C:\Users\Asus\Documents\MAESTRIA_PUCP\CICLO-2\TECNICAS_DATA_MINING\TRABAJO_FINAL\mask-rcnn\Mask_RCNN\mask_rcnn_coco.h5


In [22]:
# Image Augmentation
# Right/Left flip 50% of the time
augmentation = imgaug.augmenters.Fliplr(0.5)

In [23]:
# Training - Stage 1
print("Training network heads")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=2,#40
            layers='heads',
            augmentation=augmentation)

Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: C:\Users\Asus\Documents\MAESTRIA_PUCP\CICLO-2\TECNICAS_DATA_MINING\TRABAJO_FINAL\mask-rcnn\Mask_RCNN\logs\coco20221112T1616\mask_rcnn_coco_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistrib

C:\Users\Asus\.conda\envs\gpu-environ\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\Asus\.conda\envs\gpu-environ\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\Asus\.conda\envs\gpu-environ\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
1000/1000 [==============================] - 1491s 1s/step - loss: 0.5321 - rpn_class_loss: 0.0018 - rpn_bbox_loss: 0.1089 - mrcnn_class_loss: 0.2675 - mrcnn_bbox_loss: 0.0680 - mrcnn_mask_loss: 0.0859 - val_loss: 1.3701 - val_rpn_class_loss: 0.0145 - val_rpn_bbox_loss: 0.2824 - val_mrcnn_class_loss: 0.6443 - val_mrcnn_bbox_loss: 0.1481 - val_mrcnn_mask_loss: 0.2807

Epoch 2/2
1000/1000 [==============================] - 1479s 1s/step - loss: 0.3839 - rpn_class_loss: 0.0013 - rpn_bbox_loss: 0.0923 - mrcnn_class_loss: 0.1562 - mrcnn_bbox_loss: 0.0585 - mrcnn_mask_loss: 0.0755 - val_loss: 1.6383 - val_rpn_class_loss: 0.0470 - val_rpn_bbox_loss: 0.4811 - val_mrcnn_class_loss: 0.6004 - val_mrcnn_bbox_loss: 0.1847 - val_mrcnn_mask_loss: 0.3250


In [24]:
# Training - Stage 2
# Finetune layers from ResNet stage 4 and up
print("Fine tune Resnet stage 4 and up")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=6,#120
            layers='4+',
            augmentation=augmentation)

Fine tune Resnet stage 4 and up

Starting at epoch 2. LR=0.001

Checkpoint Path: C:\Users\Asus\Documents\MAESTRIA_PUCP\CICLO-2\TECNICAS_DATA_MINING\TRABAJO_FINAL\mask-rcnn\Mask_RCNN\logs\coco20221112T1616\mask_rcnn_coco_{epoch:04d}.h5
Selecting layers to train
res4a_branch2a         (Conv2D)
bn4a_branch2a          (BatchNorm)
res4a_branch2b         (Conv2D)
bn4a_branch2b          (BatchNorm)
res4a_branch2c         (Conv2D)
res4a_branch1          (Conv2D)
bn4a_branch2c          (BatchNorm)
bn4a_branch1           (BatchNorm)
res4b_branch2a         (Conv2D)
bn4b_branch2a          (BatchNorm)
res4b_branch2b         (Conv2D)
bn4b_branch2b          (BatchNorm)
res4b_branch2c         (Conv2D)
bn4b_branch2c          (BatchNorm)
res4c_branch2a         (Conv2D)
bn4c_branch2a          (BatchNorm)
res4c_branch2b         (Conv2D)
bn4c_branch2b          (BatchNorm)
res4c_branch2c         (Conv2D)
bn4c_branch2c          (BatchNorm)
res4d_branch2a         (Conv2D)
bn4d_branch2a          (BatchNorm)
re

C:\Users\Asus\.conda\envs\gpu-environ\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\Asus\.conda\envs\gpu-environ\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\Asus\.conda\envs\gpu-environ\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 3/6


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[400,256,28,28] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node mrcnn_mask_deconv_1/conv2d_transpose}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[loss_2/add_5/_11455]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[400,256,28,28] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node mrcnn_mask_deconv_1/conv2d_transpose}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 successful operations.
0 deri

In [ ]:
# Training - Stage 3
# Fine tune all layers
print("Fine tune all layers")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=8,#160
            layers='all',
            augmentation=augmentation)

### Evaluación

In [ ]:
class InferenceConfig(coco.CocoConfig.__class__):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0
config_inference = InferenceConfig()
config_inference.display()

In [ ]:
dataset_val = CocoDataset()
coco = dataset_val.load_coco(COCO_DIR, "val")
dataset_val.prepare()

In [ ]:
model_inf = modellib.MaskRCNN(mode="inference", config=config_inference,
                          model_dir=MODEL_DIR)

In [ ]:
weights_path = model_inf.find_last()

# Load weights
print("Loading weights ", weights_path)
model_inf.load_weights(weights_path, by_name=True)

In [ ]:
LIMIT = 30

print("Running COCO evaluation on {} images.".format(LIMIT))
evaluate_coco(model, dataset_val, coco, "bbox", limit=int(LIMIT))